<a href="https://colab.research.google.com/github/janaeshkrish/CNN_cat_and_dog/blob/master/mobilenet_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tqdm

In [ ]:
#Downloading Cats and dogs datasets
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2020-07-27 05:38:53--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.126.128, 108.177.127.128, 108.177.96.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.126.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M  83.6MB/s    in 0.8s    

2020-07-27 05:38:54 (83.6 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [ ]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.2.0'

In [ ]:
#Un zipping the dataset to the current path
dataset_path = "./cats_and_dogs_filtered.zip"

In [ ]:
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")

In [ ]:
#extracting the current path
zip_object.extractall("./")

In [ ]:
zip_object.close()

In [ ]:
dataset_path_new = "./cats_and_dogs_filtered/"

In [ ]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

In [ ]:
#Loading the pretrained model (MOBILENETV2)
IMG_SHAPE = (128, 128, 3)
#remove the top layer to add the custom head part
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

9412608/9406464 [==============================] - 0s 0us/step


In [ ]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

In [ ]:
#free the base model inorder to train 
base_model.trainable = False

In [ ]:
base_model.output

<tf.Tensor 'out_relu/Identity:0' shape=(None, 4, 4, 1280) dtype=float32>

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [ ]:
global_average_layer

<tf.Tensor 'global_average_pooling2d/Identity:0' shape=(None, 1280) dtype=float32>

In [ ]:
#setting the last layer for the model with one neuron
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

In [ ]:
# joining the basemodel and prediction layer
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
______________________________________________________________________________________________

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
#MobileNet (architecture that we use) supports: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).
# preprocessing the dataset
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [ ]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [ ]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 1000 images belonging to 2 classes.


In [ ]:
model.fit_generator(train_generator, epochs=8, validation_data=valid_generator)

Epoch 1/8
16/16 [==============================] - 32s 2s/step - loss: 0.3273 - accuracy: 0.8895 - val_loss: 0.3287 - val_accuracy: 0.8870
Epoch 2/8
16/16 [==============================] - 33s 2s/step - loss: 0.2949 - accuracy: 0.9045 - val_loss: 0.2979 - val_accuracy: 0.9070
Epoch 3/8
16/16 [==============================] - 32s 2s/step - loss: 0.2681 - accuracy: 0.9150 - val_loss: 0.2710 - val_accuracy: 0.9180
Epoch 4/8
16/16 [==============================] - 32s 2s/step - loss: 0.2450 - accuracy: 0.9255 - val_loss: 0.2492 - val_accuracy: 0.9230
Epoch 5/8
16/16 [==============================] - 32s 2s/step - loss: 0.2255 - accuracy: 0.9315 - val_loss: 0.2304 - val_accuracy: 0.9330
Epoch 6/8
16/16 [==============================] - 34s 2s/step - loss: 0.2093 - accuracy: 0.9375 - val_loss: 0.2146 - val_accuracy: 0.9370
Epoch 7/8
16/16 [==============================] - 32s 2s/step - loss: 0.1952 - accuracy: 0.9440 - val_loss: 0.2014 - val_accuracy: 0.9390
Epoch 8/8
16/16 [==========

In [ ]:
valid_loss, valid_accuracy = model.evaluate(valid_generator)

8/8 [==============================] - 9s 1s/step - loss: 0.1903 - accuracy: 0.9410


In [ ]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.9409999847412109


In [51]:
train_generator.class_indices

{'cats': 0, 'dogs': 1}

In [59]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_finetune.h5")
